### Что проверяем:

слова из списка сводеша сохраняют ранги своих соседей независимо от модели

слова из списка сводеша демонстрируют большую устойчивость к набору соседей, чем случайные слова

слова из списка сводеша сохраняют расстояния к частотным словам языка независимо от модели.


In [1]:
# coding: utf-8

'''
This module is formatting  user config with facts and entities,
creating entity index and producing all variants for entity-match
'''

import pymorphy2
import gensim

import re
import os
from tqdm import tqdm
from operator import itemgetter
from scipy import stats
import random
from functools import lru_cache

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [2]:
def open_model(m):
    if m.endswith('.vec'):
        model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
    elif m.endswith('.bin'):
        model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
    else:
        model = gensim.models.Word2Vec.load(m)
    return model

In [3]:
@lru_cache(maxsize=50)
def get_morph(token):
    return morph.parse(token)

In [4]:
def get_tagged(word):
    pos_dic = {"NOUN":["NOUN","PROPN"],    "ADJF":["ADJ"],    "ADJS":["ADJ"],    "COMP":["ADJ","ADV"],    "VERB":["VERB"],    "INFN":["VERB"],    "PRTF":["ADJ"],    "PRTS":["VERB"],    "GRND":["VERB"],    "NUMR":["NUM"],    "ADVB":["ADV"],    "NPRO":["PRON"],    "PRED":["ADV"],    "PREP":["ADP"],    "CONJ":["CCONJ","SCONJ"],    "PRCL":["PART"],    "INTJ":["INTJ"]}
    p = get_morph(word)[0]
    try:
        pos = pos_dic[p.tag.POS]
    except:
        return []
    lemma = p.normal_form
    return [lemma+'_'+tag for tag in pos]

In [5]:
def check_in_model(word, model):
    tag_list = get_tagged(word)
    for tag in tag_list:
        #тут реальное наличие
        if tag in model:
            return True
    return False

In [6]:
def get_svodesh_range(word, model, svodesh):
    syns = {}
    tag = get_tagged(word)
    
    if tag and tag[0] in model:
        syns = {s:model.similarity(tag[0],s) for s in svodesh}
    return sorted(syns.items(), key=itemgetter(1), reverse=True)

In [37]:
WDIR = r'/media/mi_air/0F0B7DDE62EEA81E/vector'
mname = r'araneum_upos_skipgram_300_2_2018.vec'
mpath = os.path.join(WDIR, mname)

In [38]:
model = open_model(mpath)

In [13]:
rus_svodesh_here = [get_tagged(i)[0] for i in rus_svodesh2 if get_tagged(i) and get_tagged(i)[0] in model.vocab ]

In [14]:
#присутствующие в модели слова
len(rus_svodesh_here)

173

In [15]:
get_svodesh_range('мама', model, rus_svodesh_here)

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('мать_NOUN', 0.70530075),
 ('муж_NOUN', 0.66201216),
 ('жена_NOUN', 0.652348),
 ('отец_NOUN', 0.5994032),
 ('женщина_NOUN', 0.5448224),
 ('мужчина_NOUN', 0.4588908),
 ('спать_VERB', 0.4059834),
 ('думать_VERB', 0.39870393),
 ('костя_NOUN', 0.39396182),
 ('смеяться_VERB', 0.39379144),
 ('человек_NOUN', 0.37384513),
 ('знать_VERB', 0.35448375),
 ('собака_NOUN', 0.35296828),
 ('грудь_NOUN', 0.35241392),
 ('умирать_VERB', 0.3406965),
 ('сказать_VERB', 0.33579656),
 ('сидеть_VERB', 0.33464593),
 ('жить_VERB', 0.32761335),
 ('ночь_NOUN', 0.32260218),
 ('ходить_VERB', 0.322582),
 ('сосать_VERB', 0.31954628),
 ('маленький_ADJ', 0.31852502),
 ('пить_VERB', 0.3162434),
 ('бояться_VERB', 0.31456605),
 ('живот_NOUN', 0.30812234),
 ('петь_VERB', 0.30691093),
 ('шить_VERB', 0.3030794),
 ('сердце_NOUN', 0.29385206),
 ('слышать_VERB', 0.29216233),
 ('прийти_VERB', 0.290725),
 ('кусать_VERB', 0.28655526),
 ('бросать_VERB', 0.2831743),
 ('приходить_VERB', 0.27893466),
 ('день_NOUN', 0.2750632),
 ('мыт

In [21]:
#araneum
rset1 = list(model.vocab)

In [18]:
#rnc
rset2 = list(model.vocab)

In [23]:
#taiga
rset3 = list(model.vocab)

In [31]:
intersec_words = set.intersection(set(rset1),set(rset2),set(rset3))

In [33]:
random_words = random.sample(intersec_words, 173)

In [34]:
random_words

['исполнительство_NOUN',
 'анормальный_ADJ',
 'обвешивать_VERB',
 'втыкать_VERB',
 'седакова_PROPN',
 'обязательство_NOUN',
 'вислоухий_ADJ',
 'заместительница_NOUN',
 'покреститься_VERB',
 'альпеншток_NOUN',
 'удешевление_NOUN',
 'наводка_NOUN',
 'шея_NOUN',
 'крамола_NOUN',
 'обновлять_VERB',
 'сведение_NOUN',
 'зашоренность_NOUN',
 'кроватный_ADJ',
 'молоканский_ADJ',
 'эмалированный_ADJ',
 'валерий::петрович_PROPN',
 'барбитурат_NOUN',
 'стробоскоп_NOUN',
 'флуктуация_NOUN',
 'плав_NOUN',
 'козырной_ADJ',
 'исцелитель_NOUN',
 'азы_NOUN',
 'шапокляк_NOUN',
 'криволинейный_ADJ',
 'пудовый_ADJ',
 'самодовлеющий_ADJ',
 'франкистский_ADJ',
 'усадьб_NOUN',
 'усаживать_VERB',
 'ослепительно_ADV',
 'наносить_VERB',
 'заздравный_ADJ',
 'присыхать_VERB',
 'ямполь_PROPN',
 'картеж_NOUN',
 'мамкин_ADJ',
 'чеканщик_NOUN',
 'шмелев_PROPN',
 'игольный_ADJ',
 'впредь_ADV',
 'расщепление_NOUN',
 'тесать_VERB',
 'быстроходный_ADJ',
 'пагубно_ADV',
 'отчет_NOUN',
 'бритт_NOUN',
 'вице-консул_NOUN',
 

# эксперимент 1 - получаем 100 ближайших соседей по каждой модели для каждого слова

по аранеа
по новостям
по нкря
по вики

затем для каждого слова считаем % пересечения соседей по моделям

In [39]:
dic_svodesh = {'news':{}, 'aranea':{},'taiga':{}, 'rnc':{}}
dic_random = {'news':{}, 'aranea':{},'taiga':{}, 'rnc':{}}

In [40]:
for w in rus_svodesh_here:
    if w not in dic_svodesh['aranea']:
        dic_svodesh['aranea'][w] = model.most_similar(w,topn=100)

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [41]:
for w in random_words:
    if w not in dic_random['aranea']:
        dic_random['aranea'][w] = model.most_similar(w,topn=100)

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [42]:
#rnc
mpath = os.path.join(WDIR, 'ruscorpora_upos_skipgram_300_5_2018.vec')
model = open_model(mpath)

In [43]:
for w in rus_svodesh_here:
    if w not in dic_svodesh['rnc']:
        try:
            dic_svodesh['rnc'][w] = model.most_similar(w,topn=100)
        except:
            dic_svodesh['rnc'][w] = []

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [44]:
for w in random_words:
    if w not in dic_random['rnc']:
        try:
            dic_random['rnc'][w] = model.most_similar(w,topn=100)
        except:
            print(w)
            dic_random['rnc'][w] = []

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [45]:
#taiga
mpath = os.path.join(WDIR, 'tayga_1_2.vec')
model = open_model(mpath)

In [46]:
for w in rus_svodesh_here:
    if w not in dic_svodesh['taiga']:
        try:
            dic_svodesh['taiga'][w] = model.most_similar(w,topn=100)
        except:
            dic_svodesh['taiga'][w] = []

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [47]:
for w in random_words:
    if w not in dic_random['taiga']:
        try:
            dic_random['taiga'][w] = model.most_similar(w,topn=100)
        except:
            print(w)
            dic_random['taiga'][w] = []

/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [48]:
import pandas as pd


In [49]:
def count_set(list1, list2, list3):
    set1, set2, set3 = set(list1), set(list2), set(list3)
    d = set.intersection(set1, set2, set3)
    u = set(list(set1)+list(set2)+list(set3))
    return len(d)/len(u)

In [50]:
def get_words(dic):
    return [i[0] for i in dic]

In [52]:
rows_list = []
for word in rus_svodesh_here:
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'svodesh'
        dict1['% intersection'] = count_set(get_words(dic_svodesh['taiga'][word]),get_words(dic_svodesh['aranea'][word]),get_words(dic_svodesh['rnc'][word]))
        rows_list.append(dict1)
for word in random_words:
        dict1 = {}
        dict1['word'] = word
        dict1['svodesh/random'] = 'random'
        dict1['% intersection'] = count_set(get_words(dic_random['taiga'][word]),get_words(dic_random['aranea'][word]),get_words(dic_random['rnc'][word]))
        rows_list.append(dict1)
df = pd.DataFrame(rows_list) 
df.head()

,% intersection,svodesh/random,word
0,0.139013,svodesh,несколько_ADV
1,0.047244,svodesh,немного_ADV
2,0.026923,svodesh,мало_ADV
3,0.100437,svodesh,четыре_NUM
4,0.143519,svodesh,пять_NUM


In [56]:
#svodesh
sv = df[df['svodesh/random']=='svodesh']
#random
rn = df[df['svodesh/random']=='random']

In [57]:
#манн-уитни
from scipy.stats import mannwhitneyu

In [60]:
mannwhitneyu(sorted(sv['% intersection']), sorted(rn['% intersection']), use_continuity=True, alternative='less')

MannwhitneyuResult(statistic=20997.0, pvalue=0.9999999999560165)

In [ ]:
#Получается, что нулевая гипотеза о том, что степень признака в выборках неравна (в первой меньше) неверна
#принимается альтернативная гипотеза, что в случайной выборке слов степень признака меньше

In [68]:
df.to_csv(path_or_buf='/media/mi_air/0F0B7DDE62EEA81E/vector/closest_word_intersection.csv', sep='\t')

In [ ]:
@lru_cache(maxsize=10)
def get_n_closest(word, model, num=5):
    syns = {}
    pos_dic = {"NOUN":["NOUN","PROPN"],    "ADJF":["ADJ"],    "ADJS":["ADJ"],    "COMP":["ADJ","ADV"],    "VERB":["VERB"],    "INFN":["VERB"],    "PRTF":["ADJ"],    "PRTS":["VERB"],    "GRND":["VERB"],    "NUMR":["NUM"],    "ADVB":["ADV"],    "NPRO":["PRON"],    "PRED":["ADV"],    "PREP":["ADP"],    "CONJ":["CCONJ","SCONJ"],    "PRCL":["PART"],    "INTJ":["INTJ"]}
    
    p = get_morph(word)[0]
    try:
        pos = pos_dic[p.tag.POS]
    except:
        pass
    lemma = p.normal_form
    for tag in pos:
        #тут реальное наличие
        if lemma+'_'+tag in model:
            syns[lemma+'_'+tag in model] = model[lemma+'_'+tag in model]#огр на число
    return syns

In [11]:
rus_svodesh = rus_svodesh.split('\n')
rus_svodesh = [i.split('(')[0] for i in rus_svodesh]
rus_svodesh = [i.strip() for i in rus_svodesh]

In [12]:
rus_svodesh = [i.split(', ') if ',' in i else i for i in rus_svodesh ]
rus_svodesh2 = []
for i in rus_svodesh:
    if type(i)==list:
        rus_svodesh2+=i
    else:
        rus_svodesh2.append(i)

In [25]:
len(rus_svodesh2)

228

In [10]:
rus_svodesh = '''я
    ты
    он, она, оно
    мы
    вы (мн. число)
    они
    этот, эта, это (близкий предмет)
    тот, та, то (удалённый предмет)
    здесь, тут (близко)
    там (далеко)
    кто (об одушевлённых субъектах)
    что (о неодушевлённых субъектах)
    где
    когда
    как
    не (отрицательная частица)
    всё (на свете); также допускается «все (люди)» (all)
    много (большое количество)
    несколько, немного (среднее количество) (some)
    мало (малое количество)
    другой (человек), другие (люди)
    один
    два
    три
    четыре
    пять
    большой (дом, предмет)
    длинный (предмет)
    широкий
    толстый (предмет)
    тяжёлый
    маленький
    короткий
    узкий
    тонкий (предмет)
    женщина
    мужчина
    человек
    ребёнок
    жена
    муж
    мать
    отец
    зверь (дикое животное)
    рыба
    птица
    собака
    вошь
    змея
    червь (дождевой червяк)
    дерево
    лес
    палка («ударил палкой»)
    плод (фрукт)
    семя (растения)
    лист (дерева)
    корень (растения)
    кора (дерева)
    цветок
    трава
    верёвка
    кожа
    мясо
    кровь
    кость
    жир (животный)
    яйцо
    рог
    хвост
    перо (птицы)
    волос(ы)
    голова
    ухо
    глаз
    нос
    рот
    зуб
    язык
    ноготь
    стопа, ступня
    нога (от стопы до бедра)
    колено
    рука (кисть)
    крыло
    живот (от пупка до промежности)
    кишки (внутренности)
    шея
    спина
    грудь (часть туловища)
    сердце
    печень
    пить (воду)
    есть (кушать)
    кусать (зубами); грызть
    сосать
    плевать
    рвать, блевать
    дуть (о ветре); отдельно допустимо также «дуть на свечу»
    дышать
    смеяться
    видеть
    слышать
    знать
    думать
    чувствовать запах; отдельно допустимо также «нюхать ноздрями» (smell)
    бояться (опасности)
    спать
    жить
    умирать
    убивать
    бороться, воевать (с врагом) (fight)
    охотиться (в лесу)
    ударить (однократно рукой)
    резать (предмет ножом)
    (раз)рубить (бревно топором) (split)
    воткнуть, вонзить (нож) (stab)
    царапать
    копать, рыть (яму)
    плавать («умеет плавать»)
    летать
    ходить, идти (шагом)
    приходить, прийти
    лежать (человек на земле)
    сидеть
    стоять
    повернуть («идя по дороге, повернуть направо») (turn)
    падать (вертикально вниз)
    давать
    держать (рукой)
    сжимать, давить (рукой с силой)
    тереть (многократно)
    мыть (руки)
    вытирать (пыль)
    тянуть (на себя)
    толкать (вперёд, от себя)
    бросать (камень)
    привязать; связать (верёвкой)
    шить (рубашку)
    считать (числа)
    сказать («он что-то сказал»)
    петь (песню)
    играть («дети играют»)
    плавать («дерево плавает, не тонет»)
    течь («река течёт»)
    замёрзнуть («вода замёрзла»)
    пухнуть («нога распухла»)
    солнце
    луна (полная)
    звезда
    вода
    дождь (средней силы)
    река (крупная или средняя)
    озеро
    море
    соль
    камень («бросил камень»)
    песок
    пыль
    земля (грунт)
    облако
    туман (достаточно густой)
    небо
    ветер
    снег
    лёд
    дым
    огонь
    зола, пепел (от костра)
    гореть («огонь горит»)
    дорога (грунтовая)
    гора (достаточно высокая)
    красный
    зелёный
    жёлтый
    белый
    чёрный
    ночь
    день
    год
    тёплый (о погоде)
    холодный (о погоде)
    полный (стакан)
    новый
    старый (предмет)
    хороший (качественный)
    плохой (некачественный)
    гнилой
    грязный (возможно, ребёнок)
    прямой (линия, дорога)
    круглый
    острый (нож)
    тупой (нож)
    гладкий, ровный (на ощупь)
    мокрый
    сухой
    правильный (верный)
    близкий («живёт близко»)
    далёкий («живёт далеко»)
    правый
    левый
    при, у, возле
    в («в доме»)
    с, со (вместе с кем-то)
    и
    если
    потому что
    имя (человека)'''